In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-08-30 13:30:19--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0.001s  

2024-08-30 13:30:19 (5.79 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [1]:
import json
import minsearch

In [23]:
# index = minsearch.Index(
#     text_fields=["name", "theme", "subtheme","category", "extendedData.notes", "review.title", "review.review", "review.author"],
#     keyword_fields=["themeGroup","packagingType","availability","extendedData.tags","review.rating.overall", "review.rating.parts", "review.datePosted"]
# )
index = minsearch.Index(
    text_fields=["name","theme","subtheme","category"],
    keyword_fields=["themeGroup","packagingType","availability"]
)

In [3]:
def load_from_json(filename):
    try:
        with open(filename, 'r') as f:
            data = json.load(f)
        print(f"Data successfully loaded from {filename}")
        return data
    except IOError as e:
        print(f"An error occurred while loading data from {filename}: {e}")
        return None

In [4]:
lego_doc = load_from_json("sets_reviews.json")

Data successfully loaded from sets_reviews.json


In [9]:
# Flattening the structure
flattened_lego_doc = []

for year, sets in lego_doc.items():
    flattened_lego_doc.extend(sets)

print(flattened_lego_doc[0])

{'setID': 49041, 'number': '10359', 'numberVariant': 1, 'name': '{?}', 'year': 2025, 'theme': 'Icons', 'themeGroup': 'Model making', 'category': 'Normal', 'released': False, 'image': {}, 'bricksetURL': 'https://brickset.com/sets/10359-1', 'collection': {}, 'collections': {'wantedBy': 135}, 'LEGOCom': {'US': {}, 'UK': {}, 'CA': {}, 'DE': {}}, 'rating': 0.0, 'reviewCount': 0, 'packagingType': '{Not specified}', 'availability': '{Not specified}', 'instructionsCount': 0, 'additionalImageCount': 0, 'ageRange': {}, 'dimensions': {}, 'barcode': {}, 'extendedData': {}, 'lastUpdated': '2024-03-04T15:05:35.09Z'}


In [24]:
index.fit(flattened_lego_doc)

In [28]:
from openai import OpenAI

In [29]:
client = OpenAI(api_key="")

In [26]:
def search(query):
    boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

In [37]:
prompt_template = """
You're a lego shop assistant. Answer the QUESTION based on the CONTEXT from Lego sets database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

def build_prompt(query, search_results):
   

    context = ""
    
    for doc in search_results:
        #context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
        context = context + doc["name"] + doc["bricksetURL"] + f"\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [31]:
search_results = search(query)

In [33]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [34]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [53]:
query = "what harry potter sets do you have"

In [54]:
rag(query)

'Based on our current database, we have the following Harry Potter LEGO sets available:\n\n1. Harry Potter Minifigure - [HPSDCC1-1](https://brickset.com/sets/HPSDCC1-1)\n2. Harry Potter Mosaic - [6268521-1](https://brickset.com/sets/6268521-1)\n3. Harry Potter Minifigure Collection - [5005254-1](https://brickset.com/sets/5005254-1)\n4. LEGO Harry Potter Advent Calendar (75981) - [75981-1](https://brickset.com/sets/75981-1)\n5. LEGO Harry Potter Advent Calendar (76404) - [76404-1](https://brickset.com/sets/76404-1)\n6. LEGO Harry Potter Advent Calendar (76418) - [76418-1](https://brickset.com/sets/76418-1)\n7. LEGO Harry Potter Advent Calendar (76390) - [76390-1](https://brickset.com/sets/76390-1)\n8. LEGO Harry Potter Advent Calendar (75964) - [75964-1](https://brickset.com/sets/75964-1)\n9. Mini Harry Potter Knight Bus - [4695-1](https://brickset.com/sets/4695-1)\n10. Harry Potter & Hermione Granger - [76393-1](https://brickset.com/sets/76393-1)'